### web scraping (só pelo o id)

In [3]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

In [ ]:
import time
def url_get(i):
    url_list= [ ]
    for id in range(243,(243+i)):
        url = "https://www.ncbi.nlm.nih.gov/nuccore/NG_009{}".format( id )
        url_list.append(url)
    return url_list
url_get(4)

In [ ]:
import requests

content = []
for url in url_get(4):
    r = requests.get(url)
    content.append(r.content)
print(content)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

    id = ""
    url = ""
    for line in lines:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id = line.attrs['content']
    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

    r2 = requests.get( url )
    r3= r2.content.decode('utf-8')
    #r3= str(r2.content)
    print (r3, '\n')

In [ ]:
#import re
#existe = re.findall(r"ORGANISM\s+.*?(?=bp)", r3, re.DOTALL)
#if existe:
    #for definition in existe:
        #m = re.match( r"ORGANISM\s+(.+)", definition, re.DOTALL )
        #print( re.sub(r'\s+', ' ', m.group(1) ) )

### web scraping (por condição)

In [11]:
import time
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    #print (a)

import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
#print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
else:
    print (n_genes)

    


escolha o que quer pesquisar: IRS
escolha o nº de genes que quer obter (máximo 20): 20
['8223391', '64880', '3376', '45785', '408438', '111506503', '39258', '125265521', '110376740', '25467', '10458', '55023', '100890677', '399388', '112802', '83946', '495836', '394823', '105148', '186242']


### Buscar Links dos genes:

In [ ]:
import time
def url_get_id(i):
    url_list= [ ]
    for id in n_genes:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
url_get_id(n_genes)

In [ ]:
import requests

content_id = []
for url in url_get_id(n_genes):
    r = requests.get(url)
    content_id.append(r.content)
#print(content_id)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content_id:
    soup_id = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines_id = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

    id_id = ""
    url_id = ""
    for line in lines_id:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id_id = line.attrs['content']
    if id:
        url_id = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id_id)

    r2 = requests.get(url_id)
    r4= r2.content.decode('utf-8')
    print (r4, '\n')

### Buscar ids:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    print(info.id)

### Buscar quantidade de cds e a sua localização:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
position=0
pos=[]
for info in records:
    #print(info.id)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            featcds.append(i)
            cds= i
            a= str(cds)
            print (len(a))
            position=i
            pos.append(info.features[position].location)
            print (pos)

### Buscar Ids de proteinas

In [12]:
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT)

['CAA43397.1', 'AAA03418.1', 'CAA68632.1', 'CAA68633.1', 'AAA31796.1', 'AAA31797.1']


### Buscar Links de proteinas:

In [13]:

import time
ID_PROT
def url_get_id_p(a):
    url_list_p=[ ]
    for id_p in ID_PROT:
        url_id_p= "https://www.uniprot.org/uniprotkb?query={}".format(id_p)
        url_list_p.append(url_id_p)
    return url_list_p
url_get_id_p(len(ID_PROT))

['https://www.uniprot.org/uniprotkb?query=CAA43397.1',
 'https://www.uniprot.org/uniprotkb?query=AAA03418.1',
 'https://www.uniprot.org/uniprotkb?query=CAA68632.1',
 'https://www.uniprot.org/uniprotkb?query=CAA68633.1',
 'https://www.uniprot.org/uniprotkb?query=AAA31796.1',
 'https://www.uniprot.org/uniprotkb?query=AAA31797.1']

In [14]:
for a in ID_PROT:
    with ExPASy.get_sprot_raw(a) as handle:
            seq_record = SeqIO.read(handle, "nt")
            print(seq_record.id)
            print(seq_record.seq)
            

HTTPError: HTTP Error 400: Bad Request

In [8]:
import requests

content_id_p = []
for url in url_get_id_p(len(ID_PROT)):
    rp = requests.get(url)
    content_id_p.append(rp.content)
#print(content_id)

In [9]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content_id_p:
    soup_id_p = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines_id_p = soup_id_p.find_all('meta', {'name':"id"} )

    id_id_p= ""
    url_id_p = ""
    for line in lines_id_p:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id_id_p = line.attrs['content']
    if id_id_p:
        url_id_p = "https://rest.uniprot.org/uniprotkb/search?query={} AND (reviewed:true)&fields='id'&size=1&format=tsv".format(id_id_p)

    r7 = requests.get(url_id_p)
    r8= r7.content.decode('utf-8')
    print (r8, '\n')

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant http://?

### Buscar seq:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:50],'...',info.seq[-10:])
    print()
    seq.append(f'{info.seq[0:50]}...{info.seq[-10:]}')
    

### Buscar gene description:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    print(info.id)
    print(info.description)
    description.append(info.description)

### Buscar artigos:

In [ ]:
Ids=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    print("title:", info.get("TI", "-"))
    print('authors: ', info.get("AU", "-"))
    print("source:", info.get("SO", "-"))
    print("affiliation: ", info.get("AD", "-") ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
    print()